In [ ]:
!pip install pomegranate
!pip install pandas

In [ ]:
!conda install -c anaconda graphviz python-graphviz -y

In [ ]:
!pip install --install-option="--include-path=/srv/conda/envs/notebook/include"  --install-option="--library-path=/srv/conda/envs/notebook/lib" pygraphviz

In [ ]:
!pip install matplotlib
!pip install pillow

In [ ]:
import pygraphviz
import tempfile
import matplotlib.pyplot as plt
import matplotlib.image

In [ ]:
def plot(nw):
    G = pygraphviz.AGraph(directed=True)
    for state in nw.states:
        G.add_node(state.name, color='red')
    for parent, child in nw.edges:
        G.add_edge(parent.name, child.name)
    with tempfile.NamedTemporaryFile() as tf:
        G.draw(tf.name, format='png', prog='dot')
        img = matplotlib.image.imread(tf.name)
        plt.imshow(img)
        plt.axis('off')

## Bayesian Network

In [ ]:
from pomegranate import *

In [ ]:
attack = DiscreteDistribution( { True: 0.01, False: 0.99 } )
_attack_node = State(attack, name="attack")

In [ ]:
AV = ConditionalProbabilityTable([
    [ True, True,  0.9 ],  #TruePositive
    [ True, False, 0.1 ],  #TrueNegative
    [ False, True, 0.005 ],#FalsePositive
    [ False, False,0.995 ] #FalseNegative
    ], [attack])
_AV_node = State(AV, name="AV")

In [ ]:
FW = ConditionalProbabilityTable([
    [ True, True,  0.7 ],   #TruePositive
    [ True, False, 0.3 ],   #TrueNegative
    [ False, True, 0.0001 ],#FalsePositive
    [ False, False,0.9999 ] #FalseNegative
    ], [attack])
_FW_node = State(FW, name="FW")

In [ ]:
#Building the Bayesian Network
network = BayesianNetwork("Security Alert With Bayesian Networks")
network.add_nodes(_attack_node, _AV_node, _FW_node)
network.add_edge(_attack_node, _AV_node)
network.add_edge(_attack_node, _FW_node)
network.bake()

In [ ]:
#network.plot()
plot(network)

In [ ]:
beliefs = network.predict_proba({ 'FW':True, 'AV':True })
for state, belief in zip(network.states, beliefs):
    print(state.name, belief)

In [ ]:
beliefs = network.predict_proba({ 'FW':False, 'AV':False })
for state, belief in zip(network.states, beliefs):
    print(state.name, belief)

## Multilayer

In [ ]:
AVt = ConditionalProbabilityTable([
    [ True, True,  1.0 ],
    [ True, False, 0.0 ],
    [ False,True,  0.0 ],
    [ False,False, 1.0 ]
    ], [attack])
_AVt_node = State(AVt, name="AVt")

In [ ]:
AV = ConditionalProbabilityTable([
    [ True, True,  0.9 ],  #TruePositive
    [ True, False, 0.1 ],  #TrueNegative
    [ False, True, 0.005 ],#FalsePositive
    [ False, False,0.995 ] #FalseNegative
    ], [AVt])
_AV_node = State(AV, name="AV")

In [ ]:
WL = ConditionalProbabilityTable([
    [ True, True,  0.1 ],
    [ True, False, 0.9 ],
    [ False, True, 0.9 ],
    [ False, False,0.1 ]
    ], [AVt])
_WL_node = State(WL, name="WL")

In [ ]:
#Building the Bayesian Network
network = BayesianNetwork("Security Alert With Bayesian Networks")
network.add_nodes(_attack_node, _AVt_node, _AV_node, _WL_node, _FW_node)
network.add_edge(_attack_node, _AVt_node)
network.add_edge(_AVt_node, _AV_node)
network.add_edge(_AVt_node, _WL_node)
network.add_edge(_attack_node, _FW_node)
network.bake()

In [ ]:
#network.plot()
plot(network)

In [ ]:
beliefs = network.predict_proba({ 'FW':False, 'AV':True, 'WL':True })
for state, belief in zip(network.states, beliefs):
    print(state.name, belief)

## XSS

In [ ]:
import pandas as pd
# Ref: https://www.kaggle.com/syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning
data = pd.read_csv("XSS_dataset.csv")
data.head()

In [ ]:
data.query("Label == 1").shape[0], data.shape[0]

In [ ]:
xss_datasets = []
features = ["alert", "script", "span", "mouseover", "href", "http", "document"]
for idx, row in data.iterrows():
    s, l = row[1], row[2]
    s = s.lower()
    d = []
    for fe in features:
        d.append(fe if s.find(fe) != -1 else "no_" + fe)
    xss_datasets.append(d)

In [ ]:
network = BayesianNetwork("XSS Tree With Bayesian Networks")
network = network.from_samples(xss_datasets)

In [ ]:
for i, fe in enumerate(features):
    print(i, fe)
#network.plot()
plot(network)

In [ ]:
beliefs = network.predict_proba({  })
for state, belief in zip(network.states, beliefs):
    print(state.name, belief)

In [ ]:
beliefs = network.predict_proba({ '3':'mouseover', '0':'alert' })
for state, belief in zip(network.states, beliefs):
    print(state.name, belief)